# CLIP 기반 임베딩 생성 모델

## 목표
이미지를 512차원 벡터로 변환하여 유사도 비교에 사용

## 모델
- **백본**: CLIP (openai/clip-vit-base-patch32)
- **출력**: 512차원 L2 정규화 벡터
- **용도**: 다회용기 이미지 간 유사도 계산

## 특징
- 사전학습된 모델 사용 (학습 불필요)
- 코사인 유사도로 이미지 비교
- 빠른 추론 속도

## 1. 환경 설정

In [ ]:
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from transformers import CLIPProcessor, CLIPModel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
import os
from tqdm import tqdm

# GPU 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 2. CLIP 모델 로드

In [ ]:
# CLIP 모델 및 프로세서 로드
model_name = "openai/clip-vit-base-patch32"

print(f"Loading CLIP model: {model_name}")
model = CLIPModel.from_pretrained(model_name)
processor = CLIPProcessor.from_pretrained(model_name)

model = model.to(device)
model.eval()

print(f"✓ Model loaded successfully")
print(f"Embedding dimension: 512")

## 3. 임베딩 생성 함수

In [ ]:
def generate_embedding(image_path, model, processor, device):
    """
    이미지에서 512차원 임베딩 벡터 생성
    
    Args:
        image_path: 이미지 파일 경로
        model: CLIP 모델
        processor: CLIP 프로세서
        device: 디바이스 (cuda/cpu)
    
    Returns:
        numpy array: L2 정규화된 512차원 벡터
    """
    # 이미지 로딩
    image = Image.open(image_path).convert('RGB')
    
    # 전처리
    inputs = processor(images=image, return_tensors="pt").to(device)
    
    # 임베딩 생성
    with torch.no_grad():
        image_features = model.get_image_features(**inputs)
    
    # L2 정규화 (코사인 유사도 계산 최적화)
    image_features = image_features / image_features.norm(p=2, dim=-1, keepdim=True)
    
    # numpy 배열로 변환
    embedding = image_features.cpu().numpy().flatten()
    
    return embedding

def generate_embedding_batch(image_paths, model, processor, device, batch_size=32):
    """
    여러 이미지의 임베딩을 배치로 생성
    
    Args:
        image_paths: 이미지 파일 경로 리스트
        model: CLIP 모델
        processor: CLIP 프로세서
        device: 디바이스
        batch_size: 배치 크기
    
    Returns:
        numpy array: (N, 512) 형태의 임베딩 행렬
    """
    embeddings = []
    
    for i in tqdm(range(0, len(image_paths), batch_size), desc='Generating embeddings'):
        batch_paths = image_paths[i:i+batch_size]
        
        # 이미지 로딩
        images = [Image.open(path).convert('RGB') for path in batch_paths]
        
        # 전처리
        inputs = processor(images=images, return_tensors="pt", padding=True).to(device)
        
        # 임베딩 생성
        with torch.no_grad():
            image_features = model.get_image_features(**inputs)
        
        # L2 정규화
        image_features = image_features / image_features.norm(p=2, dim=-1, keepdim=True)
        
        embeddings.append(image_features.cpu().numpy())
    
    return np.vstack(embeddings)

## 4. 유사도 계산 함수

In [ ]:
def calculate_cosine_similarity(embedding1, embedding2):
    """
    두 임베딩 벡터 간 코사인 유사도 계산
    
    Args:
        embedding1: 첫 번째 임베딩 벡터
        embedding2: 두 번째 임베딩 벡터
    
    Returns:
        float: 코사인 유사도 (0.0 ~ 1.0)
    """
    # L2 정규화되어 있으면 내적이 곧 코사인 유사도
    similarity = np.dot(embedding1, embedding2)
    return float(similarity)

def find_most_similar(query_embedding, database_embeddings, threshold=0.7):
    """
    데이터베이스에서 가장 유사한 임베딩 찾기
    
    Args:
        query_embedding: 쿼리 임베딩 (512,)
        database_embeddings: DB 임베딩 행렬 (N, 512)
        threshold: 최소 유사도 임계값
    
    Returns:
        tuple: (가장 유사한 인덱스, 유사도) 또는 (None, 0.0)
    """
    if len(database_embeddings) == 0:
        return None, 0.0
    
    # 배치 코사인 유사도 계산
    similarities = np.dot(database_embeddings, query_embedding)
    
    # 최대값 찾기
    max_idx = np.argmax(similarities)
    max_similarity = similarities[max_idx]
    
    # 임계값 체크
    if max_similarity >= threshold:
        return int(max_idx), float(max_similarity)
    else:
        return None, float(max_similarity)

## 5. 테스트: 단일 이미지 임베딩 생성

In [ ]:
# 테스트 이미지 경로 (직접 지정)
test_image_path = '../data/test_images/tumbler1.jpg'

# 임베딩 생성
embedding = generate_embedding(test_image_path, model, processor, device)

print(f"Embedding shape: {embedding.shape}")
print(f"Embedding norm (should be ~1.0): {np.linalg.norm(embedding):.4f}")
print(f"First 10 values: {embedding[:10]}")

# 이미지 표시
img = Image.open(test_image_path)
plt.figure(figsize=(6, 6))
plt.imshow(img)
plt.axis('off')
plt.title('Test Image')
plt.show()

## 6. 유사도 테스트: 같은 물체 vs 다른 물체

In [ ]:
# 테스트 이미지 3개 준비
# - tumbler1.jpg: 기준 텀블러
# - tumbler2.jpg: 같은 텀블러 (다른 각도)
# - cup.jpg: 다른 컵

image1_path = '../data/test_images/tumbler1.jpg'
image2_path = '../data/test_images/tumbler2.jpg'  # 같은 물체
image3_path = '../data/test_images/cup.jpg'       # 다른 물체

# 임베딩 생성
emb1 = generate_embedding(image1_path, model, processor, device)
emb2 = generate_embedding(image2_path, model, processor, device)
emb3 = generate_embedding(image3_path, model, processor, device)

# 유사도 계산
sim_same = calculate_cosine_similarity(emb1, emb2)
sim_diff = calculate_cosine_similarity(emb1, emb3)

print(f"Similarity (same object, different angle): {sim_same:.4f}")
print(f"Similarity (different object): {sim_diff:.4f}")
print(f"\nDifference: {sim_same - sim_diff:.4f}")

# 시각화
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(Image.open(image1_path))
axes[0].set_title('Reference Image')
axes[0].axis('off')

axes[1].imshow(Image.open(image2_path))
axes[1].set_title(f'Same Object\nSimilarity: {sim_same:.3f}')
axes[1].axis('off')

axes[2].imshow(Image.open(image3_path))
axes[2].set_title(f'Different Object\nSimilarity: {sim_diff:.3f}')
axes[2].axis('off')

plt.tight_layout()
plt.show()

## 7. 데이터셋에서 cup_code별 임베딩 생성

이제 `convert_labelstudio_to_dataset.py`로 생성된 데이터셋을 사용합니다.
데이터셋 구조:
```
dataset.zip
└── types/
    ├── CUP001/
    ├── CUP002/
    └── ...
```

In [ ]:
import zipfile
from collections import defaultdict

# 데이터셋 경로 설정
dataset_zip_path = '../dataset_output/dataset_20251110_120000.zip'  # 실제 파일명으로 변경
types_dir = '../data/types'  # 압축 해제 위치

# ZIP 파일에서 types/ 디렉토리만 추출
if os.path.exists(dataset_zip_path):
    print(f"Extracting types/ from {dataset_zip_path}...")
    
    with zipfile.ZipFile(dataset_zip_path, 'r') as zip_ref:
        # types/ 디렉토리 내의 파일만 추출
        types_files = [f for f in zip_ref.namelist() if f.startswith('types/')]
        
        for file in types_files:
            zip_ref.extract(file, '../data/')
    
    print(f"✓ Extracted {len(types_files)} files")
else:
    print(f"⚠ Dataset not found: {dataset_zip_path}")
    print("Please run convert_labelstudio_to_dataset.py first with --include-types option")

# cup_code별로 이미지 경로 수집
cup_code_images = defaultdict(list)

if os.path.exists(types_dir):
    for cup_code in os.listdir(types_dir):
        cup_dir = os.path.join(types_dir, cup_code)
        
        if os.path.isdir(cup_dir):
            images = [os.path.join(cup_dir, f) for f in os.listdir(cup_dir) 
                     if f.endswith(('.jpg', '.png', '.jpeg'))]
            cup_code_images[cup_code] = images
    
    print(f"\nFound {len(cup_code_images)} cup codes:")
    for cup_code, images in sorted(cup_code_images.items()):
        print(f"  {cup_code}: {len(images)} images")
else:
    print(f"⚠ Types directory not found: {types_dir}")

## 8. cup_code별 임베딩 생성 및 저장

In [ ]:
# cup_code별로 임베딩 생성
cup_code_embeddings = {}

for cup_code, image_paths in tqdm(cup_code_images.items(), desc='Processing cup codes'):
    if len(image_paths) > 0:
        # 배치 임베딩 생성
        embeddings = generate_embedding_batch(image_paths, model, processor, device, batch_size=8)
        
        # 평균 임베딩 계산 (대표 임베딩)
        mean_embedding = embeddings.mean(axis=0)
        # 정규화
        mean_embedding = mean_embedding / np.linalg.norm(mean_embedding)
        
        cup_code_embeddings[cup_code] = {
            'mean_embedding': mean_embedding,
            'all_embeddings': embeddings,
            'image_paths': image_paths,
            'num_images': len(image_paths)
        }

print(f"\n✓ Generated embeddings for {len(cup_code_embeddings)} cup codes")
print(f"\nSummary:")
for cup_code, data in sorted(cup_code_embeddings.items()):
    print(f"  {cup_code}: {data['num_images']} images → mean embedding (512,)")
    print(f"    Norm: {np.linalg.norm(data['mean_embedding']):.4f}")

## 9. cup_code 간 유사도 행렬 시각화

In [ ]:
import seaborn as sns

if len(cup_code_embeddings) > 1:
    # cup_code 리스트와 평균 임베딩 행렬 생성
    cup_codes = sorted(cup_code_embeddings.keys())
    mean_embeddings = np.array([cup_code_embeddings[cc]['mean_embedding'] for cc in cup_codes])
    
    # 유사도 행렬 계산
    similarity_matrix = np.dot(mean_embeddings, mean_embeddings.T)
    
    # 시각화
    plt.figure(figsize=(12, 10))
    sns.heatmap(similarity_matrix, annot=True, fmt='.3f', cmap='coolwarm', 
                xticklabels=cup_codes,
                yticklabels=cup_codes,
                vmin=0, vmax=1)
    plt.title('Cup Code Similarity Matrix (Mean Embeddings)')
    plt.tight_layout()
    plt.show()
    
    # 가장 유사한 cup_code 쌍 찾기
    print("\nMost similar cup code pairs:")
    for i in range(len(cup_codes)):
        for j in range(i+1, len(cup_codes)):
            sim = similarity_matrix[i, j]
            if sim > 0.7:  # 임계값 이상만 출력
                print(f"  {cup_codes[i]} ↔ {cup_codes[j]}: {sim:.4f}")
else:
    print("Need at least 2 cup codes for similarity matrix")

## 10. 실전 시나리오: 촬영 이미지로 cup_code 매칭

In [ ]:
# 시나리오:
# 1. DB에 cup_code별 대표 임베딩 저장
# 2. 사용자가 촬영한 이미지로 어떤 cup_code인지 매칭

# 쿼리 이미지 선택 (데이터셋에서 임의로 하나 선택)
if len(cup_code_embeddings) > 0:
    # 첫 번째 cup_code의 첫 번째 이미지를 쿼리로 사용
    test_cup_code = list(cup_code_embeddings.keys())[0]
    test_image_path = cup_code_embeddings[test_cup_code]['image_paths'][0]
    
    print(f"Test query image: {test_image_path}")
    print(f"Expected cup_code: {test_cup_code}")
    
    # 쿼리 임베딩 생성
    query_embedding = generate_embedding(test_image_path, model, processor, device)
    
    # DB 임베딩 준비 (cup_code별 평균 임베딩)
    db_cup_codes = list(cup_code_embeddings.keys())
    db_embeddings = np.array([cup_code_embeddings[cc]['mean_embedding'] for cc in db_cup_codes])
    
    # 매칭
    matched_idx, similarity = find_most_similar(query_embedding, db_embeddings, threshold=0.7)
    
    print("\n=== Matching Result ===")
    if matched_idx is not None:
        matched_cup_code = db_cup_codes[matched_idx]
        print(f"✓ Matched cup_code: {matched_cup_code}")
        print(f"  Similarity: {similarity:.4f}")
        print(f"  Correct: {'Yes' if matched_cup_code == test_cup_code else 'No'}")
    else:
        print(f"✗ No match found")
        print(f"  Highest similarity: {similarity:.4f}")
    
    # 상위 3개 결과 출력
    print("\nTop 3 matches:")
    similarities = np.dot(db_embeddings, query_embedding)
    top3_indices = np.argsort(similarities)[::-1][:3]
    
    for rank, idx in enumerate(top3_indices, 1):
        print(f"  {rank}. {db_cup_codes[idx]}: {similarities[idx]:.4f}")
    
    # 이미지 표시
    img = Image.open(test_image_path)
    plt.figure(figsize=(6, 6))
    plt.imshow(img)
    plt.axis('off')
    plt.title(f'Query Image\nExpected: {test_cup_code}\nMatched: {db_cup_codes[matched_idx] if matched_idx is not None else "None"}')
    plt.show()
else:
    print("No cup_code embeddings available")

## 11. 임계값 분석

In [ ]:
# 다양한 임계값에서 매칭 결과 분석
thresholds = np.arange(0.5, 1.0, 0.05)

if len(registered_embeddings) > 0 and os.path.exists(query_path):
    results = []
    
    for threshold in thresholds:
        matched_idx, similarity = find_most_similar(query_embedding, registered_embeddings, threshold=threshold)
        results.append({
            'threshold': threshold,
            'matched': matched_idx is not None,
            'similarity': similarity
        })
    
    # 시각화
    plt.figure(figsize=(10, 6))
    plt.plot(thresholds, [r['similarity'] for r in results], 'b-', linewidth=2, label='Similarity Score')
    plt.axhline(y=0.7, color='r', linestyle='--', label='User Threshold (0.7)')
    plt.axhline(y=0.75, color='orange', linestyle='--', label='Admin Threshold (0.75)')
    plt.xlabel('Threshold')
    plt.ylabel('Similarity')
    plt.title('Threshold Analysis')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
    
    print("\nRecommended thresholds:")
    print(f"  - User registered: 0.70 (덜 엄격)")
    print(f"  - Admin standard: 0.75 (더 엄격)")

## 12. 성능 벤치마크

In [ ]:
import time

if os.path.exists(test_image_path):
    # 단일 이미지 추론 속도
    num_runs = 100
    
    start_time = time.time()
    for _ in range(num_runs):
        _ = generate_embedding(test_image_path, model, processor, device)
    end_time = time.time()
    
    avg_time = (end_time - start_time) / num_runs * 1000  # ms
    
    print(f"Single image embedding generation:")
    print(f"  Average time: {avg_time:.2f} ms")
    print(f"  Throughput: {1000/avg_time:.1f} images/sec")
    
    # 유사도 계산 속도
    emb1 = generate_embedding(test_image_path, model, processor, device)
    emb2 = generate_embedding(test_image_path, model, processor, device)
    
    start_time = time.time()
    for _ in range(10000):
        _ = calculate_cosine_similarity(emb1, emb2)
    end_time = time.time()
    
    avg_time_sim = (end_time - start_time) / 10000 * 1000000  # μs
    
    print(f"\nCosine similarity calculation:")
    print(f"  Average time: {avg_time_sim:.2f} μs")
    print(f"  Throughput: {1000000/avg_time_sim:.0f} comparisons/sec")

## 13. 저장 및 로드 테스트

In [ ]:
# cup_code별 평균 임베딩을 JSON으로 저장
import json

save_dir = '../models/weights'
os.makedirs(save_dir, exist_ok=True)

# 평균 임베딩만 저장 (DB용)
embeddings_db = {}
for cup_code, data in cup_code_embeddings.items():
    embeddings_db[cup_code] = data['mean_embedding'].tolist()

# JSON 저장
json_path = os.path.join(save_dir, 'cup_code_embeddings.json')
with open(json_path, 'w') as f:
    json.dump(embeddings_db, f, indent=2)

print(f"✓ Saved {len(embeddings_db)} cup code embeddings to: {json_path}")

# 전체 데이터도 numpy로 저장 (분석용)
full_data_path = os.path.join(save_dir, 'cup_code_embeddings_full.npz')
np.savez_compressed(
    full_data_path,
    cup_codes=list(cup_code_embeddings.keys()),
    mean_embeddings=np.array([data['mean_embedding'] for data in cup_code_embeddings.values()]),
    num_images=np.array([data['num_images'] for data in cup_code_embeddings.values()])
)

print(f"✓ Saved full data to: {full_data_path}")

# 로드 테스트
with open(json_path, 'r') as f:
    loaded_embeddings = json.load(f)

print(f"\nLoad test:")
print(f"  Loaded {len(loaded_embeddings)} cup codes")
print(f"  Sample cup_code: {list(loaded_embeddings.keys())[0]}")
print(f"  Embedding dimension: {len(loaded_embeddings[list(loaded_embeddings.keys())[0]])}")

## 14. FastAPI 통합용 함수

In [ ]:
def generate_embedding_from_bytes(image_bytes, model, processor, device):
    """
    바이트 데이터에서 직접 임베딩 생성 (FastAPI 통합용)
    
    Args:
        image_bytes: 이미지 바이트 데이터
        model: CLIP 모델
        processor: CLIP 프로세서
        device: 디바이스
    
    Returns:
        list: 512차원 임베딩 벡터 (리스트 형태)
    """
    from io import BytesIO
    
    # 바이트 → PIL Image
    image = Image.open(BytesIO(image_bytes)).convert('RGB')
    
    # 전처리
    inputs = processor(images=image, return_tensors="pt").to(device)
    
    # 임베딩 생성
    with torch.no_grad():
        image_features = model.get_image_features(**inputs)
    
    # L2 정규화
    image_features = image_features / image_features.norm(p=2, dim=-1, keepdim=True)
    
    # 리스트로 변환 (JSON 직렬화 가능)
    embedding = image_features.cpu().numpy().flatten().tolist()
    
    return embedding

# 테스트
if os.path.exists(test_image_path):
    with open(test_image_path, 'rb') as f:
        image_bytes = f.read()
    
    embedding_list = generate_embedding_from_bytes(image_bytes, model, processor, device)
    
    print(f"Embedding type: {type(embedding_list)}")
    print(f"Embedding length: {len(embedding_list)}")
    print(f"First 5 values: {embedding_list[:5]}")

## 요약

### CLIP 임베딩 모델 특징
- **차원**: 512
- **정규화**: L2 norm = 1.0
- **유사도**: 코사인 유사도 (내적 계산)
- **추론 속도**: ~300ms/image (GPU)

### 새로운 데이터셋 구조
이제 `convert_labelstudio_to_dataset.py --include-types`로 생성된 데이터셋을 사용합니다:
```
dataset.zip
└── types/
    ├── CUP001/
    ├── CUP002/
    └── ...
```

### cup_code별 임베딩 생성
1. 각 cup_code의 모든 이미지에서 임베딩 생성
2. 평균 임베딩 계산 (대표 벡터)
3. L2 정규화
4. JSON으로 저장 → DB 또는 API에서 사용

### 저장된 파일
- `cup_code_embeddings.json`: cup_code → 평균 임베딩 (512차원)
- `cup_code_embeddings_full.npz`: 전체 데이터 (분석용)

### 권장 임계값
- **사용자 등록 다회용기**: 0.70 (덜 엄격)
- **관리자 표준 DB**: 0.75 (더 엄격)

### 워크플로우
1. Label Studio에서 어노테이션 → Export JSON
2. `convert_labelstudio_to_dataset.py --include-types` 실행 → `types/` 생성
3. 이 노트북 실행 → cup_code별 임베딩 생성
4. `cup_code_embeddings.json`을 DB에 로드
5. FastAPI에서 촬영 이미지 → 임베딩 → DB 매칭

### 다음 단계
1. FastAPI 서버에 통합 (`ai-server/models/embedding.py`)
2. 실전 데이터로 임계값 조정
3. 데이터베이스에 임베딩 저장 및 검색 최적화